In [1]:

import os
import sys
import json
import time
from typing import Annotated, TypedDict, List
from pathlib import Path
from dotenv import load_dotenv

# LangGraph Imports
from langgraph.graph import StateGraph, END

# LangChain Imports
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import CrossEncoder

# Vector DB & Memory
from pinecone import Pinecone
from pymongo import MongoClient

# Keyword Search & Processing
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import pypdf
from langchain_text_splitters import RecursiveCharacterTextSplitter
import nltk

# Load environment
load_dotenv()
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

# Download NLTK data (if needed)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('punkt_tab')

print("LangGraph & Libraries Loaded")

LangGraph & Libraries Loaded


In [2]:
# Load configs from previous phases
configs = {}
for phase in range(1, 7):
    try:
        with open(project_root / f"phase{phase}_config.json", 'r') as f:
            configs[f"phase{phase}"] = json.load(f)
    except FileNotFoundError:
        print(f" Warning: phase{phase}_config.json not found")

# Initialize LLM & Embeddings
llm = ChatGroq(
    model_name=configs['phase4']['llm_model'], 
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

embeddings = HuggingFaceEmbeddings(model_name=configs['phase2']['embedding_model'])
reranker = CrossEncoder(configs['phase5']['cross_encoder'])

print(" Configs & Models Loaded")

 Configs & Models Loaded


In [3]:
# 1. Pinecone (Vector DB)
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
disease_index = pc.Index(configs['phase2']['disease_index'])
scheme_index = pc.Index(configs['phase2']['scheme_index'])

# 2. MongoDB (Memory) - Use env variable directly with SSL fix
MONGODB_URI = os.getenv("MONGODB_URI", "mongodb://localhost:27017")

mongo_connected = False
try:
    import certifi
    
    # Use certifi's CA bundle for SSL
    if "mongodb+srv" in MONGODB_URI or "mongodb.net" in MONGODB_URI:
        mongo_client = MongoClient(
            MONGODB_URI,
            tlsCAFile=certifi.where(),
            serverSelectionTimeoutMS=5000
        )
    else:
        mongo_client = MongoClient(MONGODB_URI)
    
    db = mongo_client[configs['phase3']['database']]
    conversations_col = db[configs['phase3']['collections']['conversations']]
    
    # Test connection
    mongo_client.admin.command('ping')
    mongo_connected = True
    print(" Connected to Pinecone & MongoDB")
    
except Exception as e:
    print(f"⚠️ MongoDB connection issue: {e}")
    print("   Continuing without memory (stateless mode)")
    conversations_col = None

 Connected to Pinecone & MongoDB


In [4]:
# Build BM25 Indices
print(" Rebuilding BM25 Indices...")

def load_and_process_pdf(pdf_name, index_prefix):
    pdf_path = project_root / "data" / pdf_name
    reader = pypdf.PdfReader(pdf_path)
    text = "".join([p.extract_text() for p in reader.pages])
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=configs['phase1']['chunking']['size'],
        chunk_overlap=configs['phase1']['chunking']['overlap'],
        length_function=len
    )
    chunks = splitter.split_text(text)
    
    docs = []
    for i, content in enumerate(chunks):
        docs.append({
            "id": f"{index_prefix}_{i}",
            "content": content,
            "metadata": {"source": pdf_name}
        })
    return docs, BM25Okapi([word_tokenize(d['content'].lower()) for d in docs])

# Build Indices
disease_docs, bm25_disease = load_and_process_pdf("CitrusPlantPestsAndDiseases.pdf", configs['phase2']['disease_index'])
scheme_docs, bm25_scheme = load_and_process_pdf("GovernmentSchemes.pdf", configs['phase2']['scheme_index'])

print(" BM25 Indices Ready")

 Rebuilding BM25 Indices...
 BM25 Indices Ready


In [5]:
# TypedDict for Agent State
class AgentState(TypedDict):
    question: str
    chat_history: str
    intent: str
    documents: List[dict]
    answer: str

In [6]:
#  NODE 1: INTENT CLASSIFIER 
def intent_node(state: AgentState):
    print(" Analyzing Intent...")
    
    system_prompt = """
    You are an intent classifier for an agriculture bot.
    Classify the query into exactly one of these categories:
    - disease (crop pests, diseases, symptoms, treatments)
    - scheme (government schemes, subsidies, insurance, loans)
    - hybrid (how schemes help with diseases)
    - unclear (greetings, off-topic, nonsense)
    
    Return ONLY the category name in lowercase.
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt), 
        ("user", f"History: {state['chat_history']}\nQuery: {state['question']}")
    ])
    chain = prompt | llm | StrOutputParser()
    
    intent = chain.invoke({}).strip().lower()
    
    # Fallback cleanup
    valid_intents = ["disease", "scheme", "hybrid", "unclear"]
    if intent not in valid_intents:
        intent = "unclear"
        
    return {"intent": intent}


#  NODE 2: RETRIEVER (Hybrid + Rerank) 
def retrieval_node(state: AgentState):
    print(f"📂 Retrieving for intent: {state['intent']}...")
    query = state['question']
    intent = state['intent']
    
    docs = []
    
    # Helper for Hybrid Search
    def run_hybrid(index, bm25, raw_docs, source_name):
        results = []
        
        # 1. Vector Search
        vec_res = index.query(vector=embeddings.embed_query(query), top_k=5, include_metadata=True)
        for m in vec_res.matches:
            # Normalize structure: always use nested metadata
            results.append({
                "id": m.id,
                "content": m.metadata.get('text', ''),
                "metadata": {"source": m.metadata.get('source', source_name)}
            })
        
        # 2. Keyword Search
        tokens = word_tokenize(query.lower())
        scores = bm25.get_scores(tokens)
        top_n = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]
        for i in top_n:
            if scores[i] > 0:
                results.append(raw_docs[i])  # Already has correct structure
        
        return results

    # Route to correct indexes
    if intent in ['disease', 'hybrid']:
        docs.extend(run_hybrid(disease_index, bm25_disease, disease_docs, "CitrusPlantPestsAndDiseases.pdf"))
    if intent in ['scheme', 'hybrid']:
        docs.extend(run_hybrid(scheme_index, bm25_scheme, scheme_docs, "GovernmentSchemes.pdf"))
    
    # Deduplicate
    unique_docs = {d['content']: d for d in docs}.values()
    
    # Rerank
    if not unique_docs: return {"documents": []}
    
    pairs = [[query, d['content']] for d in unique_docs]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(unique_docs, scores), key=lambda x: x[1], reverse=True)
    
    # Keep top 4
    return {"documents": [d[0] for d in ranked[:4]]}


# NODE 3: GENERATOR
def generation_node(state: AgentState):
    print(" Generating Answer...")
    
    if not state['documents']:
        return {"answer": "I could not find any relevant documents to answer your question."}
    
    context_str = "\n\n".join([f"Source: {d['metadata']['source']}\n{d['content']}" for d in state['documents']])
    
    system_prompt = """
    You are an expert Agricultural Assistant.
    Answer the user's question using ONLY the context provided.
    Always cite the Source Documents at the end of your answer.
    If the context is empty, say you don't know.
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", f"Context:\n{context_str}\n\nHistory:\n{state['chat_history']}\n\nQuestion: {state['question']}")
    ])
    
    chain = prompt | llm | StrOutputParser()
    return {"answer": chain.invoke({})}


#  NODE 4: CLARIFICATION 
def clarification_node(state: AgentState):
    print(" Asking for Clarification...")
    return {"answer": "I am not sure if you are asking about a crop disease or a government scheme. Could you please clarify or provide more details?"}

In [7]:
workflow = StateGraph(AgentState)

#  Adding  Nodes
workflow.add_node("classifier", intent_node)
workflow.add_node("retriever", retrieval_node)
workflow.add_node("generator", generation_node)
workflow.add_node("clarifier", clarification_node)

# Entry Point
workflow.set_entry_point("classifier")

#  Add Conditional Routing
def route_intent(state):
    if state["intent"] == "unclear":
        return "clarifier"
    return "retriever"

workflow.add_conditional_edges(
    "classifier",
    route_intent,
    {
        "clarifier": "clarifier",
        "retriever": "retriever"
    }
)

# Linear Edges
workflow.add_edge("retriever", "generator")
workflow.add_edge("generator", END)
workflow.add_edge("clarifier", END)

#  Compile
app = workflow.compile()
print(" Agent Compiled!")

 Agent Compiled!


In [9]:
# Helper to run the graph
def run_agent(question, session_id="test_session"):
    # Get history from MongoDB (with error handling)
    chat_history = ""
    if conversations_col is not None:
        try:
            history_doc = conversations_col.find_one({"session_id": session_id})
            if history_doc:
                chat_history = "\n".join([f"{m['role']}: {m['content']}" for m in history_doc['messages'][-4:]])
        except Exception as e:
            print(f" Could not fetch history: {e}")
    
    inputs = {
        "question": question,
        "chat_history": chat_history,
        "intent": "",
        "documents": [],
        "answer": ""
    }
    
    print(f"\n USER: {question}")
    result = app.invoke(inputs)
    print(f" BOT: {result['answer']}")
    
    # Save to MongoDB (with error handling)
    if conversations_col is not None:
        try:
            conversations_col.update_one(
                {"session_id": session_id},
                {"$push": {"messages": {"$each": [
                    {"role": "user", "content": question, "timestamp": time.time()},
                    {"role": "assistant", "content": result['answer'], "timestamp": time.time()}
                ]}}},
                upsert=True
            )
        except Exception as e:
            print(f" Could not save to MongoDB: {e}")

In [10]:
# Test 1: Disease
run_agent("What are symptoms of Citrus Canker?", "demo_1")

# Test 2: Scheme
run_agent("How much subsidy for Drip Irrigation?", "demo_1")

# Test 3: Unclear (Should trigger Clarifier)
run_agent("tell me something random", "demo_1")


 USER: What are symptoms of Citrus Canker?
 Analyzing Intent...
📂 Retrieving for intent: disease...
 Generating Answer...
 BOT: The symptoms of Citrus Canker include: 
1. Corky brown lesions with yellow halos on leaves, fruit, twigs; 
2. Premature leaf/fruit drop.

Source: CitrusPlantPestsAndDiseases.pdf [31, 33]

 USER: How much subsidy for Drip Irrigation?
 Analyzing Intent...
📂 Retrieving for intent: scheme...
 Generating Answer...
 BOT: 
fit
Rs. 4,50,000
approx
Source: GovernmentSchemes.pdf

 USER: tell me something random
 Analyzing Intent...
 Asking for Clarification...
 BOT: I am not sure if you are asking about a crop disease or a government scheme. Could you please clarify or provide more details?
